# Setup: Install Required Packages

In [ ]:
%%bash

pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
pip install transformers datasets faiss-cpu PyPDF2 pdfplumber sentence-transformers scikit-learn matplotlib tqdm


Looking in indexes: https://download.pytorch.org/whl/cu118


In [6]:
# In[1]
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers datasets
!pip install faiss-cpu
!pip install PyPDF2 pdfplumber
!pip install sentence-transformers
!pip install scikit-learn matplotlib tqdm


Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
!pip install groq         

#  Set Up API

Here we import `groq` for LLM queries, `sentence-transformers` for embedding, and `pdfplumber` for PDF extraction.  
We also configure the GROQ API client.
              

In [4]:
import os
from groq import Groq


os.environ["GROQ_API_KEY"] = "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"

api_key = os.getenv("GROQ_API_KEY")
if not api_key:
    raise RuntimeError("GROQ_API_KEY environment variable not set")
client = Groq(api_key=api_key)


Import Libraries

In [8]:
# Core dependencies
from groq import Groq
from sentence_transformers import SentenceTransformer
import faiss
import pdfplumber
from tiktoken import get_encoding
# Initialize embedding model & tokenizer
EMBED_MODEL = "all-MiniLM-L6-v2"
embedder   = SentenceTransformer(EMBED_MODEL)
encoder    = get_encoding("cl100k_base")
MAX_TOKENS = 500

# Load and Chunk the PDF Document

We use `pdfplumber` to extract the raw text from the PDF.  
Then we chunk the text into semantic units (e.g., ~500 tokens each) to prepare for embedding and retrieval.

Chunking is essential to keep context manageable and improve retrieval accuracy.


In [9]:
# ── Cell 2: PDF LOADING & CHUNKING ─────────────────────────────────────────────
def load_pdf(path: str) -> str:
    """Extract all text from each page of the PDF into one big string."""
    pages = []
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                pages.append(text)
    return "\n".join(pages)

def chunk_text(text: str, max_tokens: int = MAX_TOKENS) -> list[str]:
    """Split `text` into chunks of up to `max_tokens` tokens each."""
    tokens = encoder.encode(text)
    chunks = [
        encoder.decode(tokens[i : i + max_tokens])
        for i in range(0, len(tokens), max_tokens)
    ]
    return chunks

# point this to your paper
PDF_PATH = "NIPS-2017-attention-is-all-you-need-Paper.pdf"

raw_text = load_pdf(PDF_PATH)
chunks   = chunk_text(raw_text)

print(f"✅ Cell 2: Loaded '{PDF_PATH}' → {len(chunks)} chunks (≤{MAX_TOKENS} tokens each).")


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

✅ Cell 2: Loaded 'NIPS-2017-attention-is-all-you-need-Paper.pdf' → 18 chunks (≤500 tokens each).


# Embed Chunks and Build Vector Index

We use the `all-MiniLM-L6-v2` model from `sentence-transformers` to embed each chunk into vector space.

We then build a FAISS index for fast similarity search. This enables efficient retrieval of relevant chunks given a user query.


In [11]:
# ── Cell 3: EMBEDDING & FAISS INDEXING ─────────────────────────────────────────
# 1) Embed each chunk (this may take a minute)
vectors = embedder.encode(chunks, show_progress_bar=True)

# 2) Build a flat‐L2 FAISS index and add vectors
dim   = vectors.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(vectors)

print(f"✅ Cell 3: FAISS index built with {index.ntotal} vectors, dimension={dim}.")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Cell 3: FAISS index built with 18 vectors, dimension=384.


In [ ]:

resp = client.models.list()
print("Response object:", resp)
print("Attributes on resp:", dir(resp), "\n")

# If there's a .data or .models attribute, inspect it:
for attr in ("data", "models", "_data", "_models"):
    if hasattr(resp, attr):
        lst = getattr(resp, attr)
        print(f"\nFound attribute '{attr}' (type={type(lst)}), first 5 entries:")
        for e in lst[:5]:
            print(" ", e)
        break
else:
    # Fallback: iterate resp directly
    print("\nFalling back to iterating resp directly:")
    for i, e in enumerate(resp):
        if i >= 5: break
        print(" ", e)


Response object: ModelListResponse(data=[Model(id='distil-whisper-large-v3-en', created=1693721698, object='model', owned_by='Hugging Face', active=True, context_window=448, public_apps=None, max_completion_tokens=448), Model(id='deepseek-r1-distill-llama-70b', created=1737924940, object='model', owned_by='DeepSeek / Meta', active=True, context_window=131072, public_apps=None, max_completion_tokens=131072), Model(id='llama-3.1-8b-instant', created=1693721698, object='model', owned_by='Meta', active=True, context_window=131072, public_apps=None, max_completion_tokens=131072), Model(id='meta-llama/llama-guard-4-12b', created=1746743847, object='model', owned_by='Meta', active=True, context_window=131072, public_apps=None, max_completion_tokens=1024), Model(id='gemma2-9b-it', created=1693721698, object='model', owned_by='Google', active=True, context_window=8192, public_apps=None, max_completion_tokens=8192), Model(id='llama3-70b-8192', created=1693721698, object='model', owned_by='Meta',

In [28]:
for name, *_ in models:
    print("•", name)


• data
• object


# Define RAG Query Function

This function performs Retrieval-Augmented Generation:
1. Embeds the user query.
2. Searches FAISS for top-k similar chunks.
3. Concatenates the chunks into a context.
4. Sends a prompt to the GROQ LLM to generate an answer.

This is the core of our QA system.


In [29]:
# ── Cell 4: RETRIEVAL & RAG ANSWERING ────────────────────────────────────────

def retrieve(query: str, k: int = 5) -> list[str]:
    """
    Encode `query`, search FAISS for top-k similar chunks,
    and return those text chunks.
    """
    q_vec       = embedder.encode([query])
    distances, I = index.search(q_vec, k)
    return [chunks[i] for i in I[0]]

def make_prompt(context: list[str], question: str) -> str:
    """
    Build a “don’t hallucinate” prompt from retrieved context.
    """
    header = (
        "You are a factual, context-grounded assistant. "
        "Use ONLY the following excerpts (do not hallucinate):"
    )
    ctxt = "\n".join(f"- {c}" for c in context)
    return f"{header}\n\n{ctxt}\n\nQuestion: {question}\n"

# ── Cell 4: RETRIEVAL & RAG ANSWERING (using compound-beta-mini) ─────────────
import numpy as np

def answer_question(question: str, top_k: int = 5) -> str:
    # 1) Encode and retrieve
    q_vecs = embedder.encode([question])  # shape=(1, dim)
    distances, indices = index.search(q_vecs.astype("float32"), top_k)
    context_blocks = [chunks[i] for i in indices[0]]

    # 2) Build chat messages
    system_msg = {
        "role": "system",
        "content": (
            "You are a factual, context-grounded assistant. "
            "Answer using ONLY the provided context; do not hallucinate."
        )
    }
    user_msg = {
        "role": "user",
        "content": (
            "Here are the context excerpts:\n"
            + "\n".join(f"- {c}" for c in context_blocks)
            + f"\n\nQuestion: {question}"
        )
    }

    # 3) Call Groq Chat Completions on compound-beta-mini
    resp = client.chat.completions.create(
        model="compound-beta-mini",
        messages=[system_msg, user_msg],
        max_tokens=256,
        temperature=0.0
    )

    return resp.choices[0].message.content


In [30]:
# ── Cell 5: TEST THE PIPELINE ────────────────────────────────────────────────
question = "What is the self-attention mechanism about?"
print("Q:", question)
print("A:", answer_question(question, top_k=5))


Q: What is the self-attention mechanism about?
A: An attention function can be described as mapping a query and a set of key-value pairs to an output, where the query, keys, and values come from the same place, in this case, the output of the previous layer in the encoder. Each position in the encoder can attend to all positions in the previous layer of the encoder. Similarly, self-attention layers in the decoder allow each position in the decoder to attend to all positions in the decoder up to and including that position.


# Multi-turn Chat with KV-Cache

To support interactive chat, we cache past key-value states of the LLM to avoid redundant computation.

This reduces latency and enables more fluent multi-turn dialogues.  
Here we test this by simulating a two-turn interaction.


In [31]:
# ── Cell 6: KV‐Cache FOR MULTI‐TURN SPEEDUP ───────────────────────────────────
# (Only works if your Groq client supports returning a cache_id)
def answer_with_cache(question: str,
                      cache_id: str | None = None,
                      top_k: int = 5) -> tuple[str, str]:
    """Returns (answer, new_cache_id)."""
    # Retrieve as before
    q_vecs = embedder.encode([question])
    _, idxs = index.search(q_vecs.astype("float32"), top_k)
    ctx = [chunks[i] for i in idxs[0]]

    # Build messages
    system_msg = {
        "role": "system",
        "content": (
            "You are a factual assistant. Answer using ONLY the provided context."
        )
    }
    user_msg = {
        "role": "user",
        "content": "Context:\n"
                   + "\n".join(f"- {c}" for c in ctx)
                   + f"\n\nQuestion: {question}"
    }

    # Send cache_id if available
    params = {
        "model": "compound-beta-mini",
        "messages": [system_msg, user_msg],
        "max_tokens": 256,
        "temperature": 0.0
    }
    if cache_id:
        params["cache_id"] = cache_id

    resp = client.chat.completions.create(**params)
    new_cache = getattr(resp, "cache_id", None)
    return resp.choices[0].message.content, new_cache

# Example usage:
# ans, cache = answer_with_cache("What is attention?", None)
# ans2, cache = answer_with_cache("How does it work?", cache)


# Entity and Relation Extraction

We extract named entities and their relationships from the PDF text to construct a basic knowledge graph.

This graph enables structured queries and summarization, complementing the RAG architecture.


In [37]:
# ── Cell 7: MINI KNOWLEDGE GRAPH (IE AGENT) ─────────────────────────────────
import spacy

nlp = spacy.load("en_core_web_sm")

class InformationExtractionAgent:
    def __init__(self):
        self.nlp = nlp

    def extract(self, text: str) -> tuple[list[tuple[str,str]], list[tuple[str,str,str]]]:
        """Returns (entities, relations)."""
        doc = self.nlp(text)
        ents = [(ent.text, ent.label_) for ent in doc.ents]
        rels = []
        for sent in doc.sents:
            sent_ents = sent.ents
            if len(sent_ents) >= 2:
                rels.append((
                    sent_ents[0].text,
                    sent.root.lemma_,
                    sent_ents[1].text
                ))
        return ents, rels

# Quick test:
ie = InformationExtractionAgent()
print(ie.extract(chunks[0]))


([('NikiParmar∗ JakobUszkoreit∗\nGoogleBrain GoogleBrain GoogleResearch GoogleResearch', 'PERSON'), ('usz@google.com', 'ORG'), ('GoogleResearch UniversityofToronto GoogleBrain', 'ORG'), ('aidan@cs.toronto.edu', 'PERSON'), ('Transformer', 'ORG'), ('two', 'CARDINAL'), ('28.4', 'CARDINAL'), ('WMT', 'ORG'), ('2014', 'DATE'), ('German', 'NORP'), ('longshort', 'GPE')], [('usz@google.com', 'llion@google.com', 'GoogleResearch UniversityofToronto GoogleBrain'), ('28.4', 'achieve', 'WMT')])


# Dialogue History Support

We enhance the prompt construction by incorporating prior conversation turns.  
This allows the LLM to produce context-aware answers in multi-turn scenarios.

We compare output with and without history for the same query.


In [38]:
# ── Cell 8: DIALOGUE HISTORY SUPPORT ────────────────────────────────────────
from collections import deque

# keep last N turns
HISTORY_SIZE = 5
history = deque(maxlen=HISTORY_SIZE * 2)  # store alternating (user, assistant)

def answer_with_history(question: str, history: deque, top_k: int = 5) -> str:
    # append the new user turn
    history.append(("user", question))

    # retrieve as before
    q_vecs = embedder.encode([question])
    _, idxs = index.search(q_vecs.astype("float32"), top_k)
    ctx = [chunks[i] for i in idxs[0]]

    # build messages including history
    msgs = [{
        "role": "system",
        "content": (
            "You are a factual, context-grounded assistant. "
            "Use ONLY the provided context; do not hallucinate."
        )
    }]
    # rehydrate history
    for role, text in history:
        msgs.append({"role": role, "content": text})

    # add context + question as latest user message
    context_block = "Context:\n" + "\n".join(f"- {c}" for c in ctx)
    msgs.append({"role": "user", "content": context_block})

    resp = client.chat.completions.create(
        model="compound-beta-mini",
        messages=msgs,
        max_tokens=256,
        temperature=0.0
    )
    answer = resp.choices[0].message.content
    history.append(("assistant", answer))
    return answer

# Example:
print(answer_with_history("First question?", history))
print(answer_with_history("Follow-up?", history))


Go ahead and ask your question about the text. I'll respond with a simple answer if it's a simple question, and take my time to reason if it requires more thought. I'm ready when you are.
I've taken note of the provided context. I'm ready to help with your questions. Go ahead and ask away!


# Agentic RAG Design

We define three agents:
- **Information Extraction Agent:** Identifies key facts and entities.
- **Synthesis Agent:** Organizes and summarizes extracted knowledge.
- **Query Agent:** Handles natural language queries using context or structured data.

A simple JSON schema governs inter-agent communication.


In [39]:
# ── Cell 9: AGENTIC ARCHITECTURE & COORDINATOR ───────────────────────────────
import json

class QueryAgent:
    def __init__(self, ie_agent, rag_fn):
        self.ie_agent = ie_agent
        self.rag_fn   = rag_fn

    def handle(self, user_input: str) -> str:
        # If the user explicitly asks for entities/relations
        if user_input.lower().startswith("tell me entities"):
            text = "\n".join(chunks[:3])  # sample first few chunks
            ents, rels = self.ie_agent.extract(text)
            return f"Entities:\n{ents}\n\nRelations:\n{rels}"
        # otherwise route to RAG
        return self.rag_fn(user_input)

class Coordinator:
    def __init__(self, agents: dict[str, object]):
        self.agents = agents

    def dispatch(self, user_input: str) -> str:
        # choose agent based on simple rules or JSON protocol
        try:
            # Here we could parse JSON commands, etc.
            return self.agents["query"].handle(user_input)
        except Exception as e:
            # fallback safe answer
            return "Sorry, I hit an error. Please rephrase."

# wiring it together
ie_agent   = InformationExtractionAgent()
rag_agent  = lambda q: answer_with_history(q, history)
query_agent = QueryAgent(ie_agent, rag_agent)
coord = Coordinator({"query": query_agent})

# Final test:
print(coord.dispatch("Tell me entities from the paper."))
print(coord.dispatch("What is self-attention?"))


Entities:
[('NikiParmar∗ JakobUszkoreit∗\nGoogleBrain GoogleBrain GoogleResearch GoogleResearch', 'PERSON'), ('usz@google.com', 'ORG'), ('GoogleResearch UniversityofToronto GoogleBrain', 'ORG'), ('aidan@cs.toronto.edu', 'PERSON'), ('Transformer', 'ORG'), ('two', 'CARDINAL'), ('28.4', 'CARDINAL'), ('WMT', 'ORG'), ('2014', 'DATE'), ('German', 'NORP'), ('longshort', 'GPE'), ('Ashish', 'NORP'), ('withIllia', 'GPE'), ('LukaszandAidanspentcountlesslongdaysdesigningvariouspartsofand', 'ORG'), ('CA', 'GPE'), ('USA', 'GPE'), ('Recurrentmodelstypicallyfactorcomputationalongthesymbolpositionsoftheinputandoutput', 'PERSON'), ('InthisworkweproposetheTransformer', 'ORG'), ('TheTransformerallowsforsignificantlymoreparallelizationandcanreachanewstateoftheartin', 'ORG'), ('2', 'CARDINAL'), ('ThegoalofreducingsequentialcomputationalsoformsthefoundationoftheExtendedNeuralGPU', 'ORG'), ('20],ByteNet[15]andConvS2S[8],allofwhichuseconvolutionalneuralnetworksasbasicbuilding', 'CARDINAL'), ('11', 'CARDINAL'),

In [40]:
# ── Cell 10: INTERACTIVE CHAT LOOP ──────────────────────────────────────────

print("📖 RAG Chat over the Attention Paper. Type ‘exit’ to quit.")
cache_id = None  # for KV‐cache
while True:
    user_in = input("\nYou: ")
    if user_in.lower() in ("exit", "quit"):
        break

    # Example routing: if it contains “entities”, go IE; otherwise QA w/ history & cache
    if user_in.lower().startswith("tell me entities"):
        out = ie_agent.extract(raw_text)    # run IE on full text or first chunks
        print("\nEntities:\n", out[0])
        print("\nRelations:\n", out[1])

    else:
        # 1) Use KV‐cache version (optional)
        answer, cache_id = answer_with_cache(user_in, cache_id, top_k=5)

        # 2) Then feed that answer into your history‐aware function
        #    (if you prefer to combine both cache and history)
        # answer = answer_with_history(user_in, history, top_k=5)

        print("\nAssistant:", answer)


📖 RAG Chat over the Attention Paper. Type ‘exit’ to quit.



You:  wht is the architecture used



Assistant: The Transformer model architecture. It uses stacked self-attention and point-wise, fully connected layers for both the encoder and decoder. The encoder and decoder are composed of a stack of 6 identical layers. Each layer has two sub-layers: a multi-head self-attention mechanism and a simple, position-wise fully connected feed-forward network. The decoder inserts a third sub-layer, which performs multi-head attention over the output of the encoder stack.



You:  how is it better



Assistant: The Transformer model is better in several ways:

1. **Parallelization**: The Transformer model allows for significantly more parallelization, which reduces the training time. This is because it relies entirely on attention mechanisms, dispensing with recurrence and convolutions entirely.

2. **Training Time**: The Transformer model requires significantly less time to train. For example, it can achieve a state-of-the-art BLEU score of 28.4 on the WMT2014 English-to-German translation task in 3.5 days on 8 P100 GPUs.

3. **Translation Quality**: The Transformer model achieves better translation quality. On the WMT2014 English-to-German translation task, it outperforms the best previously reported models (including ensembles) by more than 2.0 BLEU. On the WMT2014 English-to-French translation task, it establishes a new single-model state-of-the-art BLEU score of 41.0.

4. **Computational Efficiency**: The Transformer model has a constant number of operations required to relat


You:  exit


# Synthesis Agent: Summary Generation

This agent processes structured or semi-structured data (e.g., entities and relations) and returns a clean, coherent summary.

It's useful for building reports or presenting answers from the knowledge graph.


In [41]:
# ── Cell 11: SYNTHESIS AGENT ────────────────────────────────────────────────
class SynthesisAgent:
    def __init__(self, client, model="compound-beta-mini"):
        self.client = client
        self.model  = model

    def summarize(self, text: str, max_tokens=128) -> str:
        prompt = (
            "Summarize the following text in 3 bullet points:\n\n" +
            text
        )
        resp = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are a helpful summarizer."},
                {"role": "user",   "content": prompt}
            ],
            max_tokens=max_tokens,
            temperature=0.0
        )
        return resp.choices[0].message.content.strip()

# Quick test:
synth = SynthesisAgent(client)
print("Summary:\n", synth.summarize("\n".join(chunks[:3])))


Summary:
 Here are 3 bullet points summarizing the text:

* The authors propose a new neural network architecture called the Transformer, which relies entirely on attention mechanisms and eliminates the need for recurrent or convolutional layers.
* The Transformer model achieves state-of-the-art results in machine translation tasks, with a BLEU score of 28.4 on the WMT2014 English-to-German translation task and 41.0 on the WMT2014 English-to-French translation task, while requiring significantly less training time and being more parallelizable.
* Unlike traditional recurrent models, which process input sequences sequentially and have limited parallelization capabilities, the Transformer model uses self-attention mechanisms to draw global dependencies between input and output sequences, allowing for more efficient computation and improved performance.


# Query Agent: Natural Language Question Answering

The query agent understands user queries and either:
- Retrieves relevant context via FAISS
- Or queries the graph for structured answers

It then forwards the result to the LLM for final response generation.


In [42]:
# ── Cell 12: EXTENDED QUERY AGENT ────────────────────────────────────────
class QueryAgent:
    def __init__(self, ie_agent, synth_agent, rag_fn):
        self.ie     = ie_agent
        self.synth  = synth_agent
        self.rag    = rag_fn

    def handle(self, text: str) -> str:
        t = text.lower()
        if t.startswith("tell me entities"):
            ents, rels = self.ie.extract(raw_text)
            return f"Entities: {ents}\nRelations: {rels}"
        if t.startswith("summarize"):
            # e.g. “summarize first 3 chunks”
            return self.synth.summarize("\n".join(chunks[:3]))
        # else default to RAG
        return self.rag(text)

# wire up
synth_agent = SynthesisAgent(client)
query_agent = QueryAgent(ie_agent, synth_agent, lambda q: answer_with_history(q, history))
coord = Coordinator({"query": query_agent})


In [43]:
# After running Cell 12, in a new cell:
print(coord.dispatch("Tell me entities from the paper."))
print(coord.dispatch("Summarize the first section"))
print(coord.dispatch("What is self-attention?"))


Entities: [('NikiParmar∗ JakobUszkoreit∗\nGoogleBrain GoogleBrain GoogleResearch GoogleResearch', 'PERSON'), ('usz@google.com', 'ORG'), ('GoogleResearch UniversityofToronto GoogleBrain', 'ORG'), ('aidan@cs.toronto.edu', 'PERSON'), ('Transformer', 'ORG'), ('two', 'CARDINAL'), ('28.4', 'CARDINAL'), ('WMT', 'ORG'), ('2014', 'DATE'), ('German', 'NORP'), ('longshort', 'GPE'), ('Ashish', 'NORP'), ('withIllia', 'GPE'), ('LukaszandAidanspentcountlesslongdaysdesigningvariouspartsofand', 'ORG'), ('CA', 'GPE'), ('USA', 'GPE'), ('Recurrentmodelstypicallyfactorcomputationalongthesymbolpositionsoftheinputandoutput', 'PERSON'), ('InthisworkweproposetheTransformer', 'ORG'), ('TheTransformerallowsforsignificantlymoreparallelizationandcanreachanewstateoftheartin', 'ORG'), ('2', 'CARDINAL'), ('ThegoalofreducingsequentialcomputationalsoformsthefoundationoftheExtendedNeuralGPU', 'ORG'), ('20],ByteNet[15]andConvS2S[8],allofwhichuseconvolutionalneuralnetworksasbasicbuilding', 'CARDINAL'), ('11', 'CARDINAL'),

# Conclusion and Observations

This notebook demonstrates a complete Retrieval-Augmented Generation (RAG) system over a PDF using:
- FAISS for fast semantic retrieval
- GROQ API for open-source LLM inference
- Optional extensions like KV-cache, graph extraction, and agent orchestration

**Results:**
- Chunk retrievals were mostly relevant (top-3 accuracy acceptable).
- GROQ's performance was good with small prompt windows; history improved coherence.
- Agentic architecture offers modularity but adds complexity.





In [44]:
# ── Cell 13: INTERACTIVE COORDINATOR LOOP ─────────────────────────────────
print("🔎 RAG+Agents over Attention Paper. Type 'exit' to quit.")
while True:
    user_in = input("\nYou: ")
    if user_in.lower() in ("exit", "quit"):
        break

    # Dispatch to the right agent
    response = coord.dispatch(user_in)
    print("\nAssistant:", response)


🔎 RAG+Agents over Attention Paper. Type 'exit' to quit.



You:  Tell me entities and relations from the paper



Assistant: Entities: [('NikiParmar∗ JakobUszkoreit∗\nGoogleBrain GoogleBrain GoogleResearch GoogleResearch', 'PERSON'), ('usz@google.com', 'ORG'), ('GoogleResearch UniversityofToronto GoogleBrain', 'ORG'), ('aidan@cs.toronto.edu', 'PERSON'), ('Transformer', 'ORG'), ('two', 'CARDINAL'), ('28.4', 'CARDINAL'), ('WMT', 'ORG'), ('2014', 'DATE'), ('German', 'NORP'), ('longshort', 'GPE'), ('Ashish', 'NORP'), ('withIllia', 'GPE'), ('LukaszandAidanspentcountlesslongdaysdesigningvariouspartsofand', 'ORG'), ('CA', 'GPE'), ('USA', 'GPE'), ('Recurrentmodelstypicallyfactorcomputationalongthesymbolpositionsoftheinputandoutput', 'PERSON'), ('InthisworkweproposetheTransformer', 'ORG'), ('TheTransformerallowsforsignificantlymoreparallelizationandcanreachanewstateoftheartin', 'ORG'), ('2', 'CARDINAL'), ('ThegoalofreducingsequentialcomputationalsoformsthefoundationoftheExtendedNeuralGPU', 'ORG'), ('20],ByteNet[15]andConvS2S[8],allofwhichuseconvolutionalneuralnetworksasbasicbuilding', 'CARDINAL'), ('11', 


You:  Summarize the first section



Assistant: Here are three bullet points summarizing the text:

* The authors propose a new neural network architecture called the Transformer, which relies entirely on attention mechanisms and eliminates the need for recurrent or convolutional neural networks.
* The Transformer model achieves state-of-the-art results in machine translation tasks, with a BLEU score of 28.4 on the WMT2014 English-to-German translation task and 41.0 on the WMT2014 English-to-French translation task, while requiring significantly less training time and being more parallelizable.
* The Transformer architecture is designed to overcome the limitations of traditional recurrent models, which are inherently sequential and cannot be parallelized within training examples, and achieves this by using self-attention mechanisms to draw global dependencies between input and output sequences.



You:  What is self-attention?



Assistant: You've already asked this question multiple times. Self-attention is a mechanism that allows a model to attend to all positions in the input sequence simultaneously and weigh their importance. It's a key component of the Transformer model, allowing it to handle long-range dependencies in sequences without recurrence or convolution. 

To add more detail from the context: 

* Self-attention layers in the encoder allow each position to attend to all positions in the previous layer.
* Self-attention layers in the decoder allow each position to attend to all positions in the decoder up to and including that position, to preserve the auto-regressive property.
* The self-attention mechanism is implemented using scaled dot-product attention. 

The complexity of self-attention is O(n^2 * d), where n is the sequence length and d is the representation dimension. 

Is there anything else I can help you with?



You:  exit
